# Email Service - Send to all consumers


## GPU Check (optional)

In [3]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: False
GPU: None


## Download dependencies


In [1]:
%pip install openai bs4 pandas requests smtp_email_sender smtplib neon-api

ERROR: Could not find a version that satisfies the requirement smtplib (from versions: none)
ERROR: No matching distribution found for smtplib


In [2]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 17.2 MB/s eta 0:00:00


## Prompt engineering

This will be used to guide LLM


### Zero shot prompt for text classification

In [4]:
text = """
# Identity
You are AI model that classifies text into one of the 3 distinct categories. You will be given text as input. That is what you need to classify into one of the categories. 
The 3 categories are **Immigration**, **Economy**, **Civil**

# Instructions
* Do not answer in a sentence at all. 
* Do not give responses with Markdown formatting, just return a one word answer which corresponds to one of the 3 categories mentioned
* Never answer in a sentence. 
* Respond using EXACTLY one word from the allowed categories.
"""

with open('political_text_classifier.txt', 'w') as f:
    f.write(text)

### Few shot prompt for summarizer 

In [5]:
text = """
# Identity
You are an AI summarizing tool that explains political legislation in clear, easy-to-understand language so the public can better understand the text.

# Instructions
* Provide clear, simple, and accurate explanations.
* Use bullet points only.
* Focus only on what the legislation contains.
* Ignore information such as authors, dates, or political commentary.
* Do not add opinions, warnings, or analysis — only summarize content.
* Keep each bullet point concise and to the point.
* Use plain language that a general audience can understand.

# Few Shot Examples

### Use the following examples to guide your summaries.

## USER
Summarize the following legislation:
"The bill establishes a statewide program for distributing emergency cooling units to residents during high-temperature events. It directs the health department to manage distribution, prioritize vulnerable populations, and create public guidance on extreme-heat safety."

## ASSISTANT
* Creates a statewide program to provide emergency cooling devices during periods of extreme heat.
* Assigns the state health department to oversee how the devices are distributed.
* Requires the department to focus distribution on groups considered vulnerable to heat-related risks.
* Instructs the department to produce public information explaining how to stay safe during high-temperature events.

## USER
Summarize the following legislation:
"The act requires all state agencies to publish annual cybersecurity readiness reports. It also mandates the creation of minimum cybersecurity standards and establishes a review board to evaluate agency compliance."

## ASSISTANT
* Requires every state agency to publish a yearly report describing its cybersecurity preparedness.
* Establishes baseline cybersecurity standards that all agencies must follow.
* Creates a review board responsible for checking whether agencies meet the required cybersecurity standards.

## USER
Summarize the following legislation: [INSERT LEGISLATION HERE]

## ASSISTANT
Generate bullet points summarizing the legislation above.
"""

with open('political_text_summarizer.txt', 'w') as f:
    f.write(text)

## Load prompts


In [6]:
with open("political_text_classifier.txt", "r", encoding="utf-8") as file:
    political_text_classifier = file.read()

In [7]:
with open("political_text_summarizer.txt", "r", encoding="utf-8") as file:
    political_text_summarizer = file.read()

## Define AI functions

This uses OpenAI SDK and GPT 4o as base LLM


### Set up OpenAI client


In [ ]:
from openai import OpenAI

# Set api key to environment variables
client = OpenAI(
    api_key="sk-***********************"
)

### Political text classification


In [9]:
def classifyText(input_text):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": political_text_classifier
            },
            {"role": "user", "content": input_text}
        ]
    )
    return response.choices[0].message.content.strip().split()[0]

### Summarizer of legislative text


In [10]:
def summarizeText(input_text: str):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": political_text_summarizer},
            {"role": "user", "content": input_text}
        ]
    )
    return response.choices[0].message.content.strip()

## Build webscraper to get NYC legislation

BeautifulSoup will be used to access HTML of NYC Legistar website. We will save the scraped data in a Pandas dataframe for further processing


### Scrape council committee meetings


In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

request_url = "https://legistar.council.nyc.gov/Calendar.aspx?Mode=This+Week"
web_html = requests.get(request_url).text
soup = BeautifulSoup(web_html, "html.parser")
table = soup.find('table', id='ctl00_ContentPlaceHolder1_gridCalendar_ctl00')

council_meetings = []

# Skip the header row which is index 0
# Only get the 2 most recent meetings
for tr in table.find_all('tr')[1:]:  
    cells = tr.find_all('td')

    committee = cells[0].get_text(strip=True)
    date = cells[1].get_text(strip=True)
    meeting_time = cells[3].get_text(strip=True)
    
    if meeting_time == "Deferred":
        continue
    
    # Get the agenda link (from the 7th column)
    meeting_detail = cells[6].find('a')
    meeting_detail_aspx = meeting_detail['href']    
    
    if len(council_meetings) < 2:
        council_meetings.append({
            'Date': date,
            'Committee': committee,
            'Meeting Details': meeting_detail_aspx
        })
    else:
        break

### View council meetings in a formatted manner


In [12]:
df = pd.DataFrame(council_meetings)
df

,Date,Committee,Meeting Details
0,11/25/2025,City Council Stated Meeting,MeetingDetail.aspx?ID=1359999&GUID=80C2CF9F-9E...
1,11/25/2025,Committee on Women and Gender Equity,MeetingDetail.aspx?ID=1355584&GUID=629C1A78-9F...


## Scrape Meeting Details + Process through AI

- Iterate through each meeting detail and iterate through each "Introduction" legislative text
- Run the text through the 2 AI helper functions


### Define all categories to store legislation summaries

In [13]:
categories = {
    "Immigration": [],
    "Economy": [],
    "Civil": []
}

### Scrap legislation document from meeting details

In [14]:
from io import BytesIO
from docx import Document

for meeting in council_meetings:
    meeting_details_url = f"https://legistar.council.nyc.gov/{meeting['Meeting Details']}"
    meeting_details_html = requests.get(meeting_details_url).text
    soup = BeautifulSoup(meeting_details_html, "html.parser")  
    table = soup.find('table', id='ctl00_ContentPlaceHolder1_gridMain_ctl00')
    legislation_file = []
    
    for tr in table.find_all('tr')[1:]:  # Skip header row
        cells = tr.find_all('td')
        file_type = cells[6].get_text(strip=True)
        if file_type != "Introduction":
            continue
        
        file_locator = cells[0].find('a')['href']
        if len(legislation_file) > 2:
            break
        
        legislation_file.append(file_locator)
    
    for file_locator in legislation_file:
        response = requests.get(f"https://legistar.council.nyc.gov/{file_locator}").text
        soup = BeautifulSoup(response, "html.parser")

        span_body = soup.find('span', id="ctl00_ContentPlaceHolder1_lblAttachments2")
        legislation_pdf_link = span_body.find_all('a')[2]['href']
        name = soup.find('span', id="ctl00_ContentPlaceHolder1_lblName2").get_text(strip=True)
        status = soup.find('span', id="ctl00_ContentPlaceHolder1_lblStatus2").get_text(strip=True)

        fetched_document = requests.get(f"https://legistar.council.nyc.gov/{legislation_pdf_link}")
        doc = Document(BytesIO(fetched_document.content))    
        legislation_text = '\n'.join([para.text for para in doc.paragraphs])

        category = classifyText(legislation_text)
        summarized_category = summarizeText(legislation_text)

        categories[category].append({
            "Name": name,
            "Status": status,
            "Summarized": summarized_category
        })

## Email to recipients


### Get recipients list
The recipients emails are stored in Neon DB

In [ ]:
from neon_api import NeonAPI

neon = NeonAPI(
    api_key="neon_api_key",
    project="next-voters",
    organization="next-voters-org"
)

recipients = neon.db.table("email_subscriptions").select("email").execute().data

### Send emails

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

SENDER_EMAIL = "nextvoters@gmail.com"
SENDER_PASSWORD = "app-password  "  # Replace with your App Password
SUBJECT = "NYC Council Meeting Legislative Summary"

html_content = "<html><body>"
html_content += f"<h2>{SUBJECT}</h2>"

for category, items in categories.items():
    html_content += f"<h3>{category}</h3>"
    if not items:
        html_content += "<p>No items in this category.</p>"
    for item in items:
        html_content += f"<h4>{item['Name']} ({item['Status']})</h4>"
        html_content += f"<div>{item['Summarized'].replace(chr(10), '<br>')}</div>"

html_content += "</body></html>"

for recipient_email in recipients:
    try:
        msg = MIMEMultipart('alternative')
        msg['Subject'] = SUBJECT
        msg['From'] = SENDER_EMAIL
        msg['To'] = recipient_email

        html_part = MIMEText(html_content, 'html')
        msg.attach(html_part)

        print(f"Connecting to SMTP server for {recipient_email}...")
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.set_debuglevel(1)
        server.starttls()

        print("Logging in...")
        server.login(SENDER_EMAIL, SENDER_PASSWORD)

        server.send_message(msg)
        print(f"✓ Email sent successfully to {recipient_email}")

        server.quit()

    except smtplib.SMTPAuthenticationError as e:
        print(f"✗ Authentication failed: {e}")
        print("\nTroubleshooting:")
        print("1. Make sure 2-Factor Authentication is enabled on your Google account")
        print("2. Generate an App Password at: https://myaccount.google.com/apppasswords")
        print("3. Use the App Password (16 characters, no spaces) instead of your regular password")

    except Exception as e:
        print(f"✗ Error sending email to {recipient_email}: {e}")

print("\nAll emails processed.")

Connecting to SMTP server for hemitvpatel@gmail.com...


send: 'ehlo [172.28.0.12]\r\n'
reply: b'250-smtp.gmail.com at your service, [35.240.231.184]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [35.240.231.184]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS'
send: 'ehlo [172.28.0.12]\r\n'
reply: b'250-smtp.gmail.com at your service, [35.240.231.184]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-AUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your

Logging in...


reply: b'235 2.7.0 Accepted\r\n'
reply: retcode (235); Msg: b'2.7.0 Accepted'
send: 'mail FROM:<nextvoters@gmail.com> size=9506\r\n'
reply: b'250 2.1.0 OK d2e1a72fcca58-7d1516f6c47sm6541606b3a.18 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.1.0 OK d2e1a72fcca58-7d1516f6c47sm6541606b3a.18 - gsmtp'
send: 'rcpt TO:<hemitvpatel@gmail.com>\r\n'
reply: b'250 2.1.5 OK d2e1a72fcca58-7d1516f6c47sm6541606b3a.18 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.1.5 OK d2e1a72fcca58-7d1516f6c47sm6541606b3a.18 - gsmtp'
send: 'data\r\n'
reply: b'354 Go ahead d2e1a72fcca58-7d1516f6c47sm6541606b3a.18 - gsmtp\r\n'
reply: retcode (354); Msg: b'Go ahead d2e1a72fcca58-7d1516f6c47sm6541606b3a.18 - gsmtp'
data: (354, b'Go ahead d2e1a72fcca58-7d1516f6c47sm6541606b3a.18 - gsmtp')
send: b'Content-Type: multipart/alternative;\r\n boundary="===============8412910446544063254=="\r\nMIME-Version: 1.0\r\nSubject: NYC Council Meeting Legislative Summary\r\nFrom: nextvoters@gmail.com\r\nTo: hemitvpatel@gmail.com\r\n\r\n--==

✓ Email sent successfully to hemitvpatel@gmail.com

All emails processed.


reply: b'250 2.0.0 OK  1764390488 d2e1a72fcca58-7d1516f6c47sm6541606b3a.18 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1764390488 d2e1a72fcca58-7d1516f6c47sm6541606b3a.18 - gsmtp'
data: (250, b'2.0.0 OK  1764390488 d2e1a72fcca58-7d1516f6c47sm6541606b3a.18 - gsmtp')
send: 'quit\r\n'
reply: b'221 2.0.0 closing connection d2e1a72fcca58-7d1516f6c47sm6541606b3a.18 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection d2e1a72fcca58-7d1516f6c47sm6541606b3a.18 - gsmtp'
